# Geberation Results

The goal of this notebook is to compare the text generations when using the traditional RAG and when usign the re-ranking with datamodels


In [1]:
import os
import random
import numpy as np
import torch

import json
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dmcr.evaluators.Rouge_L_evaluator import Rouge_L_evaluator

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
seed = 42

torch.backends.cudnn.enabled = False
# NumPy
np.random.seed(seed)
random.seed(seed)
# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/caio.rhoden/miniconda3/envs/nq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of GPUs available: 1
GPU 0: NVIDIA RTX A5000


## Load Data

In [2]:
GENERATIONS_PATH = "generations"
rag_generations = json.load(open(f"{GENERATIONS_PATH}/rag_generations.json"))
datamodels_generations = json.load(open(f"{GENERATIONS_PATH}/datamodels_generations.json"))

In [3]:
questions = pl.read_ipc("../50_test.feather")

In [5]:
weights = torch.load("datamodels/models/preview_50_L2/weights.pt")
evaluation = torch.load("datamodels/models/preview_50_L2/weights.pt")
evaluation_R2 = pl.read_ipc("datamodels/evaluations/evaluation_preview_50_L2_R2Score.feather")

/tmp/ipykernel_641369/3455172289.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load("datamodels/models/preview_50_L2/weights.pt")
/tmp/ipykernel_641369

## Results for Generations Comparison

In [34]:
results = {
    "idx": [],
    "mean_metric_rag": [],
    "mean_metric_datamodels": [],
}
evaluator = Rouge_L_evaluator()


for i in range(len(rag_generations)):
    rag_i_results = []
    datamodels_i_results = []
    results["idx"].append(i)
    for j in range(len(rag_generations[str(i)])):

        max_rag = 0
        max_datamodels = 0
        
        for res in  questions[i]["answers"].to_numpy().flatten()[0]:

            metric_rag = evaluator.evaluate(np.array([res]), np.array([str(rag_generations[str(i)][j])]))
            metric_datamodels = evaluator.evaluate(np.array([res]), np.array([str(datamodels_generations[str(i)][j])]))
            
            max_rag = max(max_rag, metric_rag[0])
            max_datamodels = max(max_datamodels, metric_datamodels[0])

        rag_i_results.append(max_rag)
        datamodels_i_results.append(max_datamodels)

    results["mean_metric_rag"].append(np.mean(rag_i_results))
    results["mean_metric_datamodels"].append(np.mean(datamodels_i_results))
    df_results = pl.DataFrame(results)
    df_results.write_ipc("results.feather")

df_results = pl.DataFrame(results)

In [8]:
rag_generations[str(i)][j]

'According to the Death Penalty Information Center, there are'

In [9]:
np.array([res])

array(['2,718'], dtype='<U5')

In [39]:
results = pl.read_ipc("results.feather")
rag_metric = results["mean_metric_rag"].to_numpy().flatten()
datamodels_metric = results["mean_metric_datamodels"].to_numpy().flatten()

In [43]:
print(np.mean(rag_metric), np.mean(datamodels_metric))

0.05067113997113996 0.05640779220779221
